<a href="https://colab.research.google.com/github/penny1xu/RESTS/blob/main/sql_server_pull.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
#import package
!pip install censusgeocode
!pip install rets
!sudo apt-get install unixodbc-dev
!pip install pyodbc
import csv
import pandas as pd
import censusgeocode as cg
import numpy as np
import sys
from rets import Session

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
unixodbc-dev is already the newest version (2.3.7).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [51]:
%%sh
curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
curl https://packages.microsoft.com/config/ubuntu/16.04/prod.list > /etc/apt/sources.list.d/mssql-release.list
sudo apt-get update
sudo ACCEPT_EULA=Y apt-get -q -y install msodbcsql17

OK
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 https://packages.microsoft.com/ubuntu/16.04/prod xenial InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/graphics-drivers/ppa

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0Warning: apt-key output should not be parsed (stdout is not a terminal)
100   983  100   983    0     0   6687      0 --:--:-- --:--:-- --:--:--  6687
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    79  100    79    0     0    612      0 --:--:-- --:--:-- --:--:--   617


In [52]:
import pyodbc 
import pandas as pd 
conn = pyodbc.connect(DRIVER = '{ODBC Driver 17 for SQL Server}',
                      SERVER = '76.184.235.171,1433',
                      DATABASE = 'RES_DB',
                      UID = 'user123',
                      PWD = 'Sunique123')

data = pd.DataFrame()
cursor = conn.cursor()
rows = cursor.execute("SELECT TOP(10) * FROM [RES_DB].[dbo].[Inventory] WHERE [IsSold] = 1")
rows = cursor.fetchall()
#for row in rows:
   # print(row)
data = pd.DataFrame([tuple(t) for t in rows]) 
data.columns =['InventoryId', 'Address', 'IsSold']
print(data)
inVid = []
for item in data['InventoryId']:
  inVid.append(item)
#conn.commit()
inVid
#data['InventoryId'] = data['InventoryId'].astype(int)  


   InventoryId                                            Address  IsSold
0            9        206 E Ridgewood Drive, Garland, Texas 75041    True
1           10            310 Colonel Drive, Garland, Texas 75043    True
2           11  403 Chancellorsville Drive, Mesquite, TX 75149...    True
3           13          723 Caladium Drive, Mesquite, Texas 75149    True
4           14          813 Gregory Ave, Bedford, TX 76022(NW WS)    True
5           15           836 Russell Ln, Bedford, TX 76022(NW WS)    True
6           17      158 Eastwood Place, Lewisville, TX 75067-4315    True
7           18        1320 Lanemar Drive, Mesquite, TX 75149-2372    True
8           21             1815 Mimosa Avenue, Plano, Texas 75074    True
9           24             2104 Holt Road, Arlington, Texas 76006    True


[9, 10, 11, 13, 14, 15, 17, 18, 21, 24]

In [53]:

def findifNull(lo_left, lo_right, la_left, la_right, research, zip):
    result = research.search(resource='Property', resource_class='Property', dmql_query='(MlsStatus = SLD), (CloseDate = 2022-01-01-2022-05-31), (Longitude = {}+), (Longitude = {}-), (Latitude = {}+), (Latitude = {}-), (PostalCode = {})'.format(lo_left, lo_right, la_left, la_right, zip))
    count = 0
    lis = []
    di = {}
    df = pd.DataFrame()
    for item in result:
        if(item['PropertyType'] == 'Residential' and item['PropertySubType'] == 'Single Family Residence'):
            df = pd.concat([df, pd.DataFrame(item.values())], axis=1, ignore_index=True)
            count += 1
            di = item
    for key in di:
        lis.append(key)
    if(count >= 5):
        df = df.T
        df.columns = lis
        return df
    else:
        lo_left -= 0.008
        lo_right += 0.008
        la_left -= 0.008
        la_right += 0.008
        return findifNull(lo_left, lo_right, la_left, la_right, research, zip)

def nullZiphouse(house_leftLa,house_rightLa,house_leftLo,house_rightLo,research,column):
  result = research.search(resource = 'Property', resource_class='Property', dmql_query='(MlsStatus = SLD), (CloseDate = 2022-01-01-2022-06-30), (Longitude = {}+), (Longitude = {}-), (Latitude = {}+), (Latitude = {}-)'.format(house_leftLo, house_rightLo, house_leftLa, house_rightLa))
  count = 0
  df = pd.DataFrame()
  for item in result: 
    if(item['PropertyType'] == 'Residential' and item['PropertySubType'] == 'Single Family Residence'):
            df = pd.concat([df, pd.DataFrame(item.values())], axis=1, ignore_index=True) #?
            count += 1         
  if(count >= 5):  
    df = df.T
    df.columns = column # 
    return df
  else:
    house_leftLo -= 0.008
    house_rightLo += 0.008
    house_leftLa -= 0.008
    house_rightLa += 0.008
    return nullZiphouse(house_leftLa, house_rightLa, house_leftLo, house_rightLo, research, column)
    
def findHouse(house_leftLa, house_rightLa, house_leftLo, house_rightLo, house_leftSqt, house_rightSqt, research, column, zip, a, b, time1, time2):
    basic = house_leftSqt
    count = 0
    df = pd.DataFrame()
    result = research.search(resource='Property', resource_class='Property', dmql_query='(MlsStatus = SLD), (CloseDate = 2022-01-01-2022-05-31), (YearBuilt = {}+), (YearBuilt = {}-), (Longitude = {}+), (Longitude = {}-), (Latitude = {}+), (Latitude = {}-), (LivingArea = {}+), (LivingArea = {}-), (PostalCode = {})'.format(time1, time2, house_leftLo, house_rightLo, house_leftLa, house_rightLa, house_leftSqt, house_rightSqt, zip))
    for item in result:
        if(item['PropertyType'] == 'Residential' and item['PropertySubType'] == 'Single Family Residence'):
            df = pd.concat([df, pd.DataFrame(item.values())], axis=1, ignore_index=True)
            count += 1
    if(count >= 5):
        df = df.T
        df.columns = column
        return df
    else:
        house_leftLa -= 0.008
        house_rightLa += 0.008
        house_leftLo -= 0.008
        house_rightLo += 0.008
        b +=1
        if(b >= 12):
          house_leftLo += 0.096
          house_rightLo -= 0.096
          house_leftLa += 0.096
          house_rightLa -= 0.096  
          return nullZiphouse(house_leftLa,house_rightLa,house_leftLo,house_rightLo,research,column)

  #Living Area
        if(a == 0):
            if(basic <= 1100):
                house_leftSqt -= 100
                house_rightSqt += 100
            elif(basic <= 2200):
                house_leftSqt -= 200
                house_rightSqt += 200
            else:
                house_leftSqt -= 300
                house_rightSqt += 300

            a += 1
        return findHouse(house_leftLa, house_rightLa, house_leftLo, house_rightLo, house_leftSqt, house_rightSqt, research, column, zip, a, b, time1, time2)

    
def findfirst(result):
    count = 0
    for item in result:
        count += 1
    if(count == 0):
        return False
    else:
        return True    
    
def get(number, finalN, code, i, rets_client, street, city, state):
    pulled_value = {}
    result = rets_client.search(resource='Property', resource_class='Property', dmql_query='(StreetNumber = {}),(StreetName = {}),(PostalCode = {})'.format(number[i],finalN[i],code[i]))
    for item in result:
        pulled_value = item
    if(len(pulled_value) != 0):
        column = []
        for key in pulled_value:
            column.append(key)
        print("\nThis value exists in Website")
  # get whole row value
  #get each value
        zip = pulled_value.get('PostalCode')
        house_leftLa = float(pulled_value.get('Latitude'))
        house_rightLa = float(pulled_value.get('Latitude')) 
        house_leftLo = float(pulled_value.get('Longitude'))
        house_rightLo = float(pulled_value.get('Longitude'))
        house_leftSqt = float(pulled_value.get('LivingArea'))
        house_rightSqt = float(pulled_value.get('LivingArea'))
        time1 = int(pulled_value.get('YearBuilt'))
        time2 = int(pulled_value.get('YearBuilt'))
        if time1 < 1990:
            time1 = 0 
            time2 = 1989
        elif time1 >=1990 and time1 <2011:
            time1 = 1990
            time2 = 2010
        elif time1 >= 2011 and time1 < 2022:
            time1 = 2011
            time2 = 2021
        else:
            time1 = 2022
            time2 = 9999
        a = 0
        b = 0
        result1 = findHouse(house_leftLa, house_rightLa, house_leftLo, house_rightLo, house_leftSqt, house_rightSqt, rets_client, column, zip, a, b, time1, time2)
        result1 = result1[['BathroomsFull','BathroomsHalf', 'BathroomsTotalDecimal', 'BathroomsTotalInteger','BedroomsTotal', 'BuildingAreaTotal', 'City', 'CloseDate', 'ClosePrice', 'CumulativeDaysOnMarket', 
   'DaysOnMarket', 'GarageSpaces', 'Latitude', 'ListingContractDate', 'ListingId', 'ListPrice', 
   'ListSource', 'LivingArea', 'Longitude', 'LotSizeAcres', 'MlsStatus', 'OwnerName', 'OwnerPhone', 'OwnerPhoneAlternative',
   'ParcelNumber', 'ParcelNumber2', 'PoolYN', 'PostalCode', 'PreviousListPrice', 'PreviousStatus', 'PropertySubType',
   'PropertyType', 'PublicRemarks', 'PurchaseContractDate', 'SchoolDistrict', 'StandardStatus', 'StatusChangeTimestamp','StateOrProvince',
   'StreetDirPrefix', 'StreetDirSuffix', 'StreetName', 'StreetNumber', 'StreetNumberNumeric', 'StreetSuffix', 'SubdivisionName',
   'USProperty_MUI', 'YearBuilt']]
        price = []
        for item in result1['ClosePrice']:
            item = float(item)
            price.append(item)
        result1['ClosePrice'] = price
        result1['FullAdress'] = result1['StreetNumber'] + " " + result1['StreetName'] + result1['City'] + ", " +result1['StateOrProvince'] + " " + result1['PostalCode']

        mean1 = result1['ClosePrice'].mean()
        top1 = list(result1['ClosePrice'])
        top1.sort(reverse = True)
        return result1.to_csv('{} {} {}.csv'.format(number[i],finalN[i],code[i])), mean1, top1[0:3]
    else:
        print('check coordinates')
        result = cg.address(street[i], city = city[i], state= state[i], zipcode = code[i])
        if(len(result) != 0):
            lo_left = result[0]['coordinates']['x']
            lo_right = result[0]['coordinates']['x']
            la_left = result[0]['coordinates']['y']
            la_right = result[0]['coordinates']['y']
            result = findifNull(lo_left, lo_right, la_left, la_right, rets_client, code[i])
            result = result[['BathroomsFull','BathroomsHalf', 'BathroomsTotalDecimal', 'BathroomsTotalInteger','BedroomsTotal', 'BuildingAreaTotal', 'City', 'CloseDate', 'ClosePrice', 'CumulativeDaysOnMarket', 
   'DaysOnMarket', 'GarageSpaces', 'Latitude', 'ListingContractDate', 'ListingId', 'ListPrice', 
   'ListSource', 'LivingArea', 'Longitude', 'LotSizeAcres', 'MlsStatus', 'OwnerName', 'OwnerPhone', 'OwnerPhoneAlternative',
   'ParcelNumber', 'ParcelNumber2', 'PoolYN', 'PostalCode', 'PreviousListPrice', 'PreviousStatus', 'PropertySubType',
   'PropertyType', 'PublicRemarks', 'PurchaseContractDate', 'SchoolDistrict', 'StandardStatus', 'StatusChangeTimestamp', 'StateOrProvince',
   'StreetDirPrefix', 'StreetDirSuffix', 'StreetName', 'StreetNumber', 'StreetNumberNumeric', 'StreetSuffix', 'SubdivisionName',
   'USProperty_MUI', 'YearBuilt']]
            price = []
            for item in result['ClosePrice']:
                item = float(item)
                price.append(item)
            result['ClosePrice'] = price
            result['FullAddress'] = result['StreetNumber'] + " " + result['StreetName'] + result['City'] + ", " +result['StateOrProvince'] + " " + result['PostalCode']
            mean = result['ClosePrice'].mean()
            top = list(result['ClosePrice'])
            top.sort(reverse = True)
            return result, mean, top[0:3]
        else:
            print('invalid address')
            mean = 0
            top = 0
            return 0, mean, top
def judge(i):
    if i % 10 == 1:
        return 'st'
    elif i % 10 == 2:
        return 'nd'
    elif i % 10 == 3:
        return 'rd'
    else:
        return 'th'


In [54]:
def main():
    login_url = 'https://ntrdd.mlsmatrix.com/rets/Login.ashx' 
    username = '0671181_NID'
    password = 'Rt$tg6jx'
    rets_client = Session(login_url, username, password)
    rets_client.login()
    code = []
    name = []
    number = []
    street = []
    city = []
    state = []
    for item in data['Address']:
      a = item.split(',')[0]
      b = item.split(',')[2]
      c = item.split(',')[1]
      street.append(a)
      city.append(c)
      numbers = a.split()[0]
      names = a.split()[1:(len(a.split()) - 1)]
      codes = b.split()[1][0:5]
      states = b.split()[0]
      state.append(states)
      code.append(codes)
      name.append(names)
      number.append(numbers)
    finalN = []
    for item in name:
      string = ""
      for items in item:
        string += items
        string += ' '
        finalN.append(string[0:len(string) - 1])            
    for i in range(len(code)):
      result, mean, top = get(number, finalN, code, i, rets_client, street, city, state)
      for j in range(len(result['FullAddress'])):
        cursor.execute("INSERT INTO FinalResult4 (id, Address, ClosePrice) VALUES (?,?,?)",
                inVid[i], 
                result['FullAddress'][j],
                int(result['ClosePrice'][j])
                )
        
      print('The mean for {}, {}, {} is {}'.format(number[i],finalN[i],code[i], mean))
      print('The top 3 for {}, {}, {} are {}'.format(number[i],finalN[i],code[i], top))
    cursor.commit()
    conn.close()
    return
       
    
if __name__ == "__main__":
    print(main())

check coordinates
The mean for 206, E, 75041 is 223667.85714285713
The top 3 for 206, E, 75041 are [285000.0, 275000.0, 275000.0]
check coordinates
The mean for 310, E Ridgewood, 75043 is 282642.7272727273
The top 3 for 310, E Ridgewood, 75043 are [370000.0, 370000.0, 332500.0]
check coordinates
The mean for 403, Colonel, 75149 is 282198.5833333333
The top 3 for 403, Colonel, 75149 are [425000.0, 400000.0, 360000.0]
check coordinates
The mean for 723, Chancellorsville, 75149 is 252392.52631578947
The top 3 for 723, Chancellorsville, 75149 are [400000.0, 385000.0, 360000.0]
check coordinates
The mean for 813, Caladium, 76022 is 311412.5
The top 3 for 813, Caladium, 76022 are [402000.0, 373000.0, 360000.0]
check coordinates
The mean for 836, Gregory, 76022 is 293784.2105263158
The top 3 for 836, Gregory, 76022 are [405000.0, 371000.0, 353000.0]
check coordinates
The mean for 158, Russell, 75067 is 327727.2727272727
The top 3 for 158, Russell, 75067 are [515000.0, 475000.0, 440000.0]
chec

# how to insert value in

---



In [56]:
#create a new table
# Import CSV


# Connect to SQL Server
conn = pyodbc.connect(DRIVER = '{ODBC Driver 17 for SQL Server}',
                      SERVER = '76.184.235.171,1433',
                      DATABASE = 'RES_DB',
                      UID = 'user123',
                      PWD = 'Sunique123')

cursor = conn.cursor()
#cursor.execute('drop table if exists FinalResult')
# Create Table
cursor.execute('''
		CREATE TABLE FinalResult4 (
			id int,
			Address nvarchar(max),
			ClosePrice int
      FOREIGN KEY (id) REFERENCES Inventory(InventoryId)
			)
               ''')
conn.commit()
conn.close()

# Insert DataFrame to Table



ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 15))



ProgrammingError: ignored

insert data

In [ ]:
from pandas.core.frame import DataFrame
# insert into sql
#data = data.values.tolist()

conn = pyodbc.connect(DRIVER = '{ODBC Driver 17 for SQL Server}',
                      SERVER = '76.184.235.171,1433',
                      DATABASE = 'RES_DB',
                      UID = 'user123',
                      PWD = 'Sunique123')
cursor = conn.cursor()
sql = "INSERT INTO [RES_DB].[dbo].[SearchResult2] (SearchID,Address,IsSold) values(?,?,?)"
cursor.executemany(sql,rows)
conn.commit()
#cursor.close()
#for index, row in df.iterrows():
#cursor.execute("INSERT INTO [RES_DB].[dbo].[SearchResult2] (SearchID,Address) values(?,?)", row.InventoryId, row.Address)


check data in sql

In [ ]:
conn = pyodbc.connect(DRIVER = '{ODBC Driver 17 for SQL Server}',
                      SERVER = '76.184.235.171,1433',
                      DATABASE = 'RES_DB',
                      UID = 'user123',
                      PWD = 'Sunique123')
cursor = conn.cursor()
cursor.execute("SELECT * FROM [RES_DB].[dbo].[SearchResult2]")
rows = cursor.fetchall()
for row in rows:
    print(row)
cursor.close()



In [ ]:
conn = pyodbc.connect(DRIVER = '{ODBC Driver 17 for SQL Server}',
                      SERVER = '76.184.235.171,1433',
                      DATABASE = 'RES_DB',
                      UID = 'user123',
                      PWD = 'Sunique123')

data = pd.DataFrame()
cursor = conn.cursor()
rows = cursor.execute("SELECT * FROM [RES_DB].[dbo].[FinalResult4]")
rows = cursor.fetchall()
for row in rows:
    print(row)
conn.close()